## Enosh Nyarige
### Assignment 3 - Web Server Access Log 

#### Data Mining

##### I took data from EDGAR's log file (https://www.sec.gov/dera/data/edgar-log-file-data-set.html)

In [1]:
import pandas as pd
from datetime import datetime
import pytz
import re

#### Data Pre-processing

- Return the string deliminated by two chars

In [2]:
def parse_str(txt):
    if txt is None:
        return '-'
    return txt[1:-1]

- If no error occured in the parse_str, return the string. Otherwise, 0

In [3]:
def parse_int(txt):
    if txt is None:
        return 0
    try:
        y = int(txt)
        return y
    except ValueError:
        return 0

- Return datetimewith timezone formatted as [day/month/year:hour:minute:second zone]

In [4]:
def parse_datetime(txt):
    try:
        dt = datetime.strptime(txt[1:-7], '%d/%b/%Y:%H:%M:%S')
        dt_tz = int(txt[-6:-3])*60+int(x[-3:-1])
        return dt.replace(tzinfo=pytz.FixedOffset(dt_tz))
    except ValueError:
        return '-'

- Apply the methods above to clean the regex in the dataset

In [5]:
convet_data = {'time': parse_datetime,
        'request': parse_str,
        'status': parse_int,
        'size': parse_int,
        'referer': parse_str,
        'user_agent': parse_str}

In [6]:
ls

 accessLog.txt   log20170630.zip   README.md
 log20170630/    logs_canvas/     'web-server access log .ipynb'


- Import the downloaded dataset and define the columns in the log

In [7]:
data = pd.read_csv('accessLog.txt',
                   header=None,
                   error_bad_lines=False,
                   sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])',
                   engine='python',
                   na_values='-',
                   converters=convet_data,
                   usecols=[0, 3, 4, 5, 6, 7, 8],
                   names=['Client_IP', 'Timestamp','pageRequested', 'code', 'pageSize', 'refereringPage','clientDevice'])              
data.head(4)

,Client_IP,Timestamp,pageRequested,code,pageSize,refereringPage,clientDevice
0,13.66.139.0,[19/Dec/2020:13:57:26 +0100],"""GET /index.php?option=com_phocagallery&view=c...",200.0,32653.0,"""-""","""Mozilla/5.0 (compatible; bingbot/2.0; +http:/..."
1,157.48.153.185,[19/Dec/2020:14:08:06 +0100],"""GET /apache-log/access.log HTTP/1.1""",200.0,233.0,"""-""","""Mozilla/5.0 (Windows NT 6.3; Win64; x64) Appl..."
2,157.48.153.185,[19/Dec/2020:14:08:08 +0100],"""GET /favicon.ico HTTP/1.1""",404.0,217.0,"""http://www.almhuette-raith.at/apache-log/acce...","""Mozilla/5.0 (Windows NT 6.3; Win64; x64) Appl..."
3,216.244.66.230,[19/Dec/2020:14:14:26 +0100],"""GET /robots.txt HTTP/1.1""",200.0,304.0,"""-""","""Mozilla/5.0 (compatible; DotBot/1.1; http://w..."


#### 1. Find 10 people who visited the site frequently. 

- I will need the IP and the client device

In [8]:
IP=set(data['Client_IP'])
print(len(IP))
devices=set(data['clientDevice'])
len(devices)
D={}
for i,row in data.iterrows():
    if(tuple((row['Client_IP'],row['clientDevice'])) in D):
        D[tuple((row['Client_IP'],row['clientDevice']))]+=1
    
    else:
        D[tuple((row['Client_IP'],row['clientDevice']))]=1

5369


In [61]:
from operator import itemgetter

In [63]:
frequentUsers = dict(sorted(D.items(), key = itemgetter(1), reverse = True)[:10]) 

In [64]:
frequentUsers

{('193.106.31.130',
  '"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0)"'): 117969,
 ('197.52.128.37',
  '"Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko"'): 40771,
 ('178.44.47.170', '"python-requests/2.25.1"'): 2774,
 ('51.210.183.78', '"curl/7.68.0"'): 2684,
 ('173.255.176.5',
  '"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:82.0) Gecko/20100101 Firefox/82.0"'): 2663,
 ('173.255.176.5',
  '"Mozilla/5.0 (compatible; 008/0.83; http://www.80legs.com/webcrawler.html) Gecko/2008032620"'): 1678,
 ('45.15.143.155',
  '"Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:28.0) Gecko/20100101 Firefox/28.0"'): 1284,
 ('45.15.143.155', '"python-requests/2.25.0"'): 642,
 ('37.120.205.149',
  '"Mozilla/4.0 (compatible; Win32; WinHttp.WinHttpRequest.5)"'): 562,
 ('173.255.176.5',
  '"Mozilla/5.0 (X11; Linux x86_64; rv:78.0) Gecko/20100101 Firefox/78.0"'): 530}

- In this case, I sorted the dictionary in a descending order, and listed the top 10 from the dictionary as the frequent 10 people who visited the site.
    - For each user, the count appended after the userAgent accounts for the metric of measuring the top and frequent visitors to the site
       
       
<!--   -->
<!--  -->
        - For instance, the top user with an IP address of 193.106.31.130 has a frequency of 117969

#### 2. Show at least five sessions and the page views per each session. 

- I will need the IP, client device, Timestamp and the requested page(s) in determining sessions

In [10]:
time=[]
page=[]
al=[]
for i in list(D)[:200]:
    for j,row in data.iterrows():
        if(i==tuple((row['Client_IP'],row['clientDevice']))):
            time.append(row['Timestamp'])
            page.append(row['pageRequested'])
            al.append(i)

In [70]:
al[:5]

[('13.66.139.0',
  '"Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)"'),
 ('13.66.139.0',
  '"Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)"'),
 ('13.66.139.0',
  '"Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)"'),
 ('157.48.153.185',
  '"Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"'),
 ('157.48.153.185',
  '"Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"')]

In [26]:
time[:5]

['[19/Dec/2020:13:57:26 +0100]',
 '[22/Jan/2021:01:57:53 +0100]',
 '[07/Feb/2021:18:59:27 +0100]',
 '[19/Dec/2020:14:08:06 +0100]',
 '[19/Dec/2020:14:08:08 +0100]']

In [27]:
page[:5]

['"GET /index.php?option=com_phocagallery&view=category&id=1:almhuette-raith&Itemid=53 HTTP/1.1"',
 '"GET /index.php?option=com_phocagallery&view=category&id=1:almhuette-raith&Itemid=53 HTTP/1.1"',
 '"GET /index.php?option=com_content&view=article&id=46&Itemid=54 HTTP/1.1"',
 '"GET /apache-log/access.log HTTP/1.1"',
 '"GET /favicon.ico HTTP/1.1"']

- Page views per each session

In [71]:
df=pd.DataFrame({'accessTime':time,'accessedPages':page,'accessClient':al})

In [72]:
df.head(10)

,accessTime,accessedPages,accessClient
0,[19/Dec/2020:13:57:26 +0100],"""GET /index.php?option=com_phocagallery&view=c...","(13.66.139.0, ""Mozilla/5.0 (compatible; bingbo..."
1,[22/Jan/2021:01:57:53 +0100],"""GET /index.php?option=com_phocagallery&view=c...","(13.66.139.0, ""Mozilla/5.0 (compatible; bingbo..."
2,[07/Feb/2021:18:59:27 +0100],"""GET /index.php?option=com_content&view=articl...","(13.66.139.0, ""Mozilla/5.0 (compatible; bingbo..."
3,[19/Dec/2020:14:08:06 +0100],"""GET /apache-log/access.log HTTP/1.1""","(157.48.153.185, ""Mozilla/5.0 (Windows NT 6.3;..."
4,[19/Dec/2020:14:08:08 +0100],"""GET /favicon.ico HTTP/1.1""","(157.48.153.185, ""Mozilla/5.0 (Windows NT 6.3;..."
5,[19/Dec/2020:14:14:26 +0100],"""GET /robots.txt HTTP/1.1""","(216.244.66.230, ""Mozilla/5.0 (compatible; Dot..."
6,[02/Jan/2021:18:39:07 +0100],"""GET /robots.txt HTTP/1.1""","(216.244.66.230, ""Mozilla/5.0 (compatible; Dot..."
7,[02/Jan/2021:19:09:15 +0100],"""GET /index.php?option=com_content&view=articl...","(216.244.66.230, ""Mozilla/5.0 (compatible; Dot..."
8,[05/Jan/2021:11:18:18 +0100],"""GET /robots.txt HTTP/1.1""","(216.244.66.230, ""Mozilla/5.0 (compatible; Dot..."
9,[05/Jan/2021:11:39:30 +0100],"""GET /apache-log/?S=A HTTP/1.1""","(216.244.66.230, ""Mozilla/5.0 (compatible; Dot..."


#### 3. Show five frequent pages which the users visit before visiting this particular web site.  

- I will need the referer page feature

In [73]:
referers = data['refereringPage'].dropna(how='all')
from urllib.parse import urlparse
ref=[]
for i in referers:       
    parsed_uri = urlparse(i.replace('"', ''))
    result = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)
    ref.append(result)
word_counter = {}
for url in ref:
    if url in word_counter:
        word_counter[url] += 1
    else:
        word_counter[url] = 1
popular_words = sorted(word_counter, key = word_counter.get, reverse = True)
top_5 = popular_words[:6]
for i in top_5:
    if(i==':///'''):
        top_5.remove(i)
for i,j in enumerate(top_5,start=1):
    print(i,"",j)

1  http://www.almhuette-raith.at/
2  http://almhuette-raith.at/
3  https://www.google.com/
4  http://simplesite.com/
5  http://www.google.com/


#### 4. Ten frequent sequential patterns or navigational patterns which the users follow using the GSP algorithm

In [74]:
Pages_visited_per_session=list(df['accessedPages'])
Pages_accessed={}
for i,row in df.iterrows():
    if(row['accessClient'] in Pages_accessed):
        Pages_accessed[row['accessClient']].append(row['accessedPages'])    
    else:
        Pages_accessed[row['accessClient']]=[row['accessedPages']]
for i in Pages_accessed:
    print('\n Client {} accessed these pages \n {} \n'.format(i,Pages_accessed[i]))
all_pages=[]
for i in Pages_accessed:
    all_pages.append(Pages_accessed[i])


 Client ('13.66.139.0', '"Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)"') accessed these pages 
 ['"GET /index.php?option=com_phocagallery&view=category&id=1:almhuette-raith&Itemid=53 HTTP/1.1"', '"GET /index.php?option=com_phocagallery&view=category&id=1:almhuette-raith&Itemid=53 HTTP/1.1"', '"GET /index.php?option=com_content&view=article&id=46&Itemid=54 HTTP/1.1"'] 


 Client ('157.48.153.185', '"Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"') accessed these pages 
 ['"GET /apache-log/access.log HTTP/1.1"', '"GET /favicon.ico HTTP/1.1"'] 


 Client ('216.244.66.230', '"Mozilla/5.0 (compatible; DotBot/1.1; http://www.opensiteexplorer.org/dotbot, help@moz.com)"') accessed these pages 
 ['"GET /robots.txt HTTP/1.1"', '"GET /robots.txt HTTP/1.1"', '"GET /index.php?option=com_content&view=article&id=50&Itemid=56 HTTP/1.1"', '"GET /robots.txt HTTP/1.1"', '"GET /apache-log/?S=A HTTP/1.1"', '"GE

In [75]:
from apyori import apriori

In [136]:
association_rules = apriori(all_pages, min_support=0.07, min_confidence=0.5, min_lift=1, min_length=3)  
association_results = list(association_rules)
support=[]
items=[]
rhs=[]
lhs=[]
con=[]
lift=[]
for i in association_results:
    support.append(i.support)
    items.append(i.items)
    rhs.append(i.ordered_statistics[0][1])
    lhs.append(i.ordered_statistics[0][0])
    con.append(i.ordered_statistics[0][2])
    lift.append(i.ordered_statistics[0][3])

In [137]:
df=pd.DataFrame({'Items':items,'Antecedent':lhs,'Precedent':rhs,'Support':support, 'Confidence':con,'Lift':lift})

In [138]:
df.head(10)

,Items,Antecedent,Precedent,Support,Confidence,Lift
0,"(""GET /index.php?option=com_contact&view=conta...",(),"(""GET /index.php?option=com_contact&view=conta...",0.86,0.86,1.0
1,"(""POST /index.php?option=com_contact&view=cont...",(),"(""POST /index.php?option=com_contact&view=cont...",0.84,0.84,1.0
2,"(""POST /index.php?option=com_contact&view=cont...",(),"(""POST /index.php?option=com_contact&view=cont...",0.84,0.84,1.0


In [159]:
s_p=df[df['Support']>0.20].sort_values('Support',ascending=False)
for i,items in df.iterrows():
    
    print("Rule: " + str(items['Antecedent']) + " -> " + str(items['Precedent']))
    print("Support: " + str(items['Support']))
    print("Confidence: " + str(items['Confidence']))
    print("Lift: " + str(items['Lift']))
    print("**"*40 + '\n')

Rule: frozenset() -> frozenset({'"GET /index.php?option=com_contact&view=contact&id=1 HTTP/1.1"'})
Support: 0.86
Confidence: 0.86
Lift: 1.0
********************************************************************************

Rule: frozenset() -> frozenset({'"POST /index.php?option=com_contact&view=contact&id=1 HTTP/1.1"'})
Support: 0.84
Confidence: 0.84
Lift: 1.0
********************************************************************************

Rule: frozenset() -> frozenset({'"POST /index.php?option=com_contact&view=contact&id=1 HTTP/1.1"', '"GET /index.php?option=com_contact&view=contact&id=1 HTTP/1.1"'})
Support: 0.84
Confidence: 0.84
Lift: 1.0
********************************************************************************



In [160]:
s_p

,Items,Antecedent,Precedent,Support,Confidence,Lift
0,"(""GET /index.php?option=com_contact&view=conta...",(),"(""GET /index.php?option=com_contact&view=conta...",0.86,0.86,1.0
1,"(""POST /index.php?option=com_contact&view=cont...",(),"(""POST /index.php?option=com_contact&view=cont...",0.84,0.84,1.0
2,"(""POST /index.php?option=com_contact&view=cont...",(),"(""POST /index.php?option=com_contact&view=cont...",0.84,0.84,1.0


#### 5. Create a graph that shows clusters of users with similar  navigational patterns